# Network analysis

## 1. Read dataset linking wikipedia articles with publications and create a bipartite graph of the relation

In [2]:
import os
import numpy as np
import pandas as pd
import networkx as nx
%matplotlib inline
import matplotlib.pyplot as plt

In [3]:
base_path = '../data/raw'
processed_path = '../data/processed'

In [4]:
# read TSV data
df = pd.read_csv(os.path.join(base_path,'enwiki.tsv'), sep='\t', parse_dates=['timestamp'],infer_datetime_format=True)

# Convert mistakenly converted type nan to string 'NaN' (wikipedia page name)
df.page_title = df.page_title.fillna("NaN")

df.head(5)

,page_id,page_title,rev_id,timestamp,type,id
0,2867096,Mu Aquilae,503137751,2012-07-19 16:08:41,doi,10.1051/0004-6361:20078357
1,2867096,Mu Aquilae,508363722,2012-08-20 22:56:21,arxiv,astro-ph/0604502
2,2867096,Mu Aquilae,508363722,2012-08-20 22:56:21,arxiv,astro-ph/0003329
3,2867096,Mu Aquilae,508363722,2012-08-20 22:56:21,arxiv,0708.1752
4,2867096,Mu Aquilae,503137751,2012-07-19 16:08:41,doi,10.1051/0004-6361:20064946


In [5]:
df1000 = df[0:1000]

In [13]:
df1000.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
page_id       1000 non-null int64
page_title    1000 non-null object
rev_id        1000 non-null int64
timestamp     1000 non-null datetime64[ns]
type          1000 non-null object
id            1000 non-null object
dtypes: datetime64[ns](1), int64(2), object(3)
memory usage: 47.0+ KB


**Create a bipartite graph**

Prepare lists of nodes and edges from the dataframe

In [6]:
# list of unique web page ids (web_page nodes)
wp_ids = df1000.page_id.unique()

# list of unique web page names
wp_titles = df1000.page_title.unique()

# list of unique publications (publication nodes)
pub_ids = df1000.id.unique()

# list of references (edges)
edges = [(page, pub) for page, pub in zip(wp_titles, pub_ids)]

In [7]:
wp_node_list = [(title, {'bipartite':'web_page', 'pid': pid,'address':'/wiki/'+title.replace(' ','_'), 'ptype':'topic','depth':0}) for pid, title in zip(wp_ids, wp_titles)]

In [8]:
pub_node_list = [(pub_id, {'bipartite':'publication'}) for pub_id in pub_ids]

Create a bipartite directed graph 

In [9]:
G = nx.DiGraph()
G.add_nodes_from(wp_node_list)#, bipartite='web_page')
G.add_nodes_from(pub_node_list)#, bipartite='publication')
G.add_edges_from(edges)

In [11]:
print(len(G.nodes()), len(G.edges()))

1261 272


Make lists of web_page and publication nodes (same as above but, now extracted directly from the graph)

In [14]:
#wpage_nodes = [node for node, data in G.nodes(data=True) if data['bipartite']=='web_page']
#pub_nodes = [node for node, data in G.nodes(data=True) if data['bipartite']=='publication']

Calculate bipartite degree centrality

In [15]:
#dcent = nx.bipartite.degree_centrality(G, wpage_nodes)

Assign degree of centrality to the two partitions and rank nodeds within each partition

In [16]:
# webpage ranking
#wpage_dcent = [(node, dcent[node]) for node in wpage_nodes]
#wpage_rank = sorted(wpage_dcent, key=lambda x: x[1], reverse=True)

# publication ranking
#pub_dcent = [(node, dcent[node]) for node in pub_nodes]
#pub_rank = sorted(pub_dcent, key=lambda x: x[1], reverse=True)

## 2. Create category hierarchy starting at articles based on Wikipedia scraping

Add category pages to the graph

In [17]:
import requests
import bs4
from collections import deque

Define a queue for pages to be explored in a breadth-first search

In [18]:
wpage_nodes = [(node, data) for node, data in G.nodes(data=True) if data['bipartite']=='web_page']

# Add all base pages to the FIFO queue
queue = deque(wpage_nodes)

Also define a set of page addresses for quick search of node presence.

In [19]:
node_titles = set([node for node, data in G.nodes(data=True) if data['bipartite']=='web_page'])

Base URLs for wiki pages and wiki API

In [20]:
wiki_url = 'http://en.wikipedia.org'
api_query = 'https://en.wikipedia.org/w/api.php?action=query&format=json&titles='

Define a function for scraping wiki pages for categories

In [21]:
def get_cats(address):
    """
    Accept a wiki page element and return a list of category elements found on the page.
    Parameters
    ----------
    address: str
             Address of a wikipedia web page (without the wiki_url base)
    Returns
    -------
    cats: List of BeautifulSoup tag objects
          Contains (not-hidden) category information for subsequent processing   
    """

    r = requests.get(wiki_url+address)

    soup = bs4.BeautifulSoup(r.text, 'html.parser')
    
    cats = soup.select('#mw-normal-catlinks ul li a')
    
    return cats 

Cycle over queue and append new nodes and edges to the graph

In [22]:
it = 0
while len(queue) > 0:

    # pop the first node in queue
    node = queue.popleft()

    # get the address of the page
    address = node[1]['address']
    depth = node[1]['depth']

    # print each 100 cycles
    if it%100 == 0:
        print('Iteration:', it, len(queue), len(G.nodes()), len(G.edges()), node[0], node[1]['ptype'])
    it += 1

    # limit the search to three levels from topic web page
    if depth < 3:

        # scrape the page and return tags for categories
        cats = get_cats(address)

        # create nodes for the categories (if new)
        for c in cats:

            # get category page title (== node identification)
            title = c.get_text()

            # create a new edge pointing from higher level category to the present one
            G.add_edge(title, node[0])

            if title not in node_titles:

                # get referenced page title
                title = c.get_text()

                # get category page address
                cat_ref = c.get('href')

                # create a new node tuple
                new_depth = depth + 1
                new_node = (title, {'address':cat_ref, 'ptype':'category', 'depth':new_depth})
                #print(new_node)

                # create a new node
                G.add_node(title, address=cat_ref, ptype='category')

                # update queue and node_titles set
                queue.append(new_node)
                node_titles.add(title)

Iteration: 0 271 1261 272 Mu Aquilae topic
Iteration: 100 803 1893 945 Wonder Nine topic
Iteration: 200 1225 2415 1509 John Grandy topic
Iteration: 300 1588 2878 2034 Rulers of Moldavia category
Iteration: 400 1797 3187 2388 Spaniels category
Iteration: 500 1935 3425 2662 Archaeological sites in South Africa category
Iteration: 600 2080 3670 2959 1980s rock song stubs category
Iteration: 700 2238 3928 3261 Baronets in the Baronetage of the United Kingdom category
Iteration: 800 2374 4164 3558 Members of the United States House of Representatives from South Carolina category
Iteration: 900 2501 4391 3830 Regents of China category
Iteration: 1000 2614 4604 4100 Third Lanark A.C. managers category
Iteration: 1100 2733 4823 4361 Modern artists category
Iteration: 1200 2867 5057 4659 Deaths from pneumonia category


ConnectionError: HTTPConnectionPool(host='en.wikipedia.org', port=80): Max retries exceeded with url: /wiki/Category:1940s_comedy_films (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x11756bef0>: Failed to establish a new connection: [Errno 65] No route to host',))

## 3. Analyze the network - degree centrality for page/category subgraphs

In [9]:
def create_subgraph(G, node):
    nodes = nx.single_source_shortest_path(G,node).keys()
    return G.subgraph(nodes)